# Lanchain OpenAI Setup

In [3]:
import langchain
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Import chat templates
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    load_prompt
)

# Caching
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

In [3]:
cacheType = 'in_memory'

if cacheType == 'in_memory':
    set_llm_cache(InMemoryCache())
elif cacheType == 'sqlite':
    set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# Set OpenAI API key and create LLM and Chat LLM. Note that key can be stored in a separate file or as an environment variable. Refer to docs.
api_key = open('./openai_key.txt').read()
llm = OpenAI(openai_api_key=api_key)
chat = ChatOpenAI(openai_api_key=api_key)

In [4]:
# Step 1 - Import parser
# Import comma separated list parser from langchain
from langchain.output_parsers import (
    CommaSeparatedListOutputParser,
    DatetimeOutputParser,
    OutputFixingParser,
    PydanticOutputParser
)
from pydantic import BaseModel, Field

In [5]:
# Step 2 - format instructions
output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [6]:
sample_response = "foo, bar, baz"
output_parser.parse(sample_response)

['foo', 'bar', 'baz']

In [7]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [8]:
request = "give me 5 characteristics of dogs"
model_request = chat_prompt.format_prompt(request=request, format_instructions=output_parser.get_format_instructions()).to_messages()

In [9]:
result = chat(model_request)
print(f'Original output: {result.content}') # result.content
print(f'Parsed output: {output_parser.parse(result.content)}')

C:\ProgramData\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Original output: loyal, playful, protective, social, intelligent
Parsed output: ['loyal', 'playful', 'protective', 'social', 'intelligent']


### Handling incorrect parse responses

There are two main ways to solve this problem:
- System prompt - Have a strong system prompt to combine with your format instructions.
- OutputFixingParser - Using a chain, send your original reply to an LLM to try to fix it

In [11]:
datetime_parser = DatetimeOutputParser()
datetime_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0617-04-19T02:46:05.097427Z, 1487-12-21T01:42:18.042091Z, 0560-10-12T08:26:06.578430Z\n\nReturn ONLY this string, no other words!"

In [12]:
dt_template = "{request}\n{format_instructions}"
dt_prompt = HumanMessagePromptTemplate.from_template(dt_template)

# Using system prompt to guide the model
dt_system_template = "You only respond with a datetime string."
dt_system_prompt = SystemMessagePromptTemplate.from_template(dt_system_template)

dt_chat_prompt = ChatPromptTemplate.from_messages([dt_prompt, dt_system_prompt])

In [13]:
dt_request = "What data was the 13th Amendment oratified in the US?"
dt_model_request = dt_chat_prompt.format_prompt(request=dt_request, format_instructions=datetime_parser.get_format_instructions()).to_messages()

In [14]:
dt_result = chat(dt_model_request, temperature=0.0)
print(f'Original output: {dt_result.content}') # result.content
print(f'Parsed output: {datetime_parser.parse(dt_result.content)}')

Original output: 1865-12-06T00:00:00.000000Z
Parsed output: 1865-12-06 00:00:00


In [15]:
# Fix misformatted output using OutputFixingParser. This will reprompt the model to try and get it to fix the output.

from langchain.output_parsers import OutputFixingParser
misformatted = dt_result.content

new_parser = OutputFixingParser.from_llm(parser=datetime_parser, llm=chat)
fixed = new_parser.parse(misformatted)

print(fixed)

1865-12-06 00:00:00


# Parsing into a class with Pydantic

In [17]:
from langchain.output_parsers import PydanticOutputParser

In [18]:
from pydantic import BaseModel, Field

In [19]:
class Scientist(BaseModel):
    
    name: str = Field(description="Name of Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [20]:
parser = PydanticOutputParser(pydantic_object=Scientist)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "Name of Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}\n```'

In [5]:
human_template = '{request}\n{format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [30]:
request = chat_prompt.format_prompt(request="Tell me about a famous scientist",
                                   format_instructions=parser.get_format_instructions()).to_messages()

In [32]:
result = chat(request, temperature=0.0)

In [36]:
parser.parse(result.content)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Brownian Motion'])

# Serializaton of prompt templates to files

In [9]:
chat_prompt.save("prompts/basic_formatted_human_template.json")

NotImplementedError: 